To Do:
- Have scaper identify and download forms in formats other than pdfs (e.g., doc and docx)
- Add more jurisdictions/states

# Load Libraries

In [1]:
import urllib.request, json 
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import pandas as pd
import time
from datetime import date
import os
from os import walk
import os.path
from os import path
import uuid

import hashlib

In [2]:
def hashme(w):
    h = hashlib.md5(w.encode('utf-8'))
    #h = hashlib.sha224(w.encode('utf-8'))
    return h.hexdigest()

In [3]:
#hashme("adsadjty")

In [4]:
def reCase(text):
    output = re.sub("(\w|\d)(_|-)(\w|\d)","\\1 \\3",text)
    output = re.sub("([a-z])([A-Z]|\d)","\\1 \\2",output)
    output = re.sub("(\d)([A-Z]|[a-z])","\\1 \\2",output)
    return output

In [8]:
import requests

from urllib.request import Request, urlopen
from urllib.parse import quote

#!pip install PyPDF2
import PyPDF2

from io import StringIO # https://stackoverflow.com/a/18284900
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage

def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb') # python three change
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text

#def download_pdf(url,path,filename):
#    filename = path+filename
#    try:
#        f = urlopen(url)
#        with open(filename, "wb") as code:
#            code.write(f.read())
#        return filename
#    except:
#        return "error"
    
def download_pdf(url,path,filename):
    try:
        filename = path+filename

        f = requests.get(url, stream=True, headers={'User-agent': 'Mozilla/5.0'})

        with open(filename, "wb") as code:
            #code.write(f.read())
            code.write(f.content)

        return convert(filename)
    except:
        return "error"

## Jump to Jurisdiction 
- [Michigan](#Download-MI-forms-and-meta-data-from)
- [Utah](#Download-UT-forms-and-meta-data-from)
- [Georgia](#Download-GA-forms-and-meta-data-from)
- [New Hampshire](#Download-NH-forms-and-meta-data-from)
- [California](#Download-CA-forms-and-meta-data-from)
- [Maine](#Download-ME-forms-and-meta-data-from)

[back to top](#Jump-to-Jurisdiction)

# Download NH forms and meta data from

https://www.courts.nh.gov/our-courts/superior-court/forms/all-forms
https://www.courts.nh.gov/our-courts/circuit-court/district-division/forms/all-forms-alphabetical-order
https://www.courts.nh.gov/our-courts/circuit-court/family-division/forms/all-forms-alphabetical-order
https://www.courts.nh.gov/our-courts/circuit-court/probate-division/forms/all-forms-alphabetical-order

[back to top](#Jump-to-Jurisdiction)

# Download GA forms and meta data from

Links on page to zip files. 
https://georgiacourts.gov/statecourt/state-court-behind-the-bench/state-court-forms/#1566241462728-b588212e-e7e2

Texas 
Illinois 
Indiana
Maryland
Minisota 


[back to top](#Jump-to-Jurisdiction)

# Download CA forms and meta data from

https://www.courts.ca.gov/allforms.htm

In [13]:
jur = "CA"
source = "www.courts.ca.gov"
site = "https://www.courts.ca.gov/allforms.htm"
form_path = "../data/raw/www.courts.ca.gov/forms/"
csv_path = "../data/raw/www.courts.ca.gov/"

try:
    files_df = pd.read_csv(csv_path+"form_data.csv")
except:
    files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"])
    files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")   
    
files_df[:1]

,id,jurisdiction,source,title,group,url,filename,downloaded


In [14]:
with urllib.request.urlopen(urllib.request.Request(site)) as url:
    content = url.read().decode()
soup = BeautifulSoup(content)

In [15]:
today = date.today().strftime("%Y-%m-%d")
body = soup.find('div', {"class": "jcc-forms-filter__form-results"})
i = 0
limit = 5000
for row in body.find_all('div', {"class": "jcc-forms-filter__form-result-content"}):
    #print(row)
    group = row.find('div', {"class": "form-number"}).text
    group = re.sub("-.*$","",group)
    title = row.find('div', {"class": "form-title"}).text
    for a in row.find_all('a', href=True):
        if (re.search("^http.*\.pdf",a['href'])):
            url = re.search(".*/([^/]*)\.pdf",a['href'], re.IGNORECASE)
            filename = url.group(1)+".pdf"
            fileurl = url.group(0)
            fileid = hashme(fileurl)
            
    status = ""
    if fileurl not in files_df["url"].values:
        print("Downloading: "+fileurl)
        if (i < limit):
            status = download_pdf(fileurl,form_path,fileid+".pdf")

        if (i < limit):
            files_df = files_df.append(pd.DataFrame([[fileid,jur,source,title,group,fileurl,filename,today]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"]), ignore_index=True,sort=False)
            i += 1
        else:
            print("skip")
        #time.sleep(1)            
            
files_df

Downloading: https://www.courts.ca.gov/documents/adopt050info.pdf
Downloading: https://www.courts.ca.gov/documents/adopt200.pdf


/Users/davidcolarusso/opt/anaconda3/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='../data/raw/www.courts.ca.gov/forms/3979f1c1c9f165ccac026b26cf20252c.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


Downloading: https://www.courts.ca.gov/documents/adopt205.pdf


/Users/davidcolarusso/opt/anaconda3/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='../data/raw/www.courts.ca.gov/forms/9f441bfaca115775970cf03550e0c78e.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


Downloading: https://www.courts.ca.gov/documents/adopt206.pdf
Downloading: https://www.courts.ca.gov/documents/adopt210.pdf
Downloading: https://www.courts.ca.gov/documents/adopt215.pdf
Downloading: https://www.courts.ca.gov/documents/adopt216.pdf


...

Downloading: https://www.courts.ca.gov/documents/wv800.pdf


/Users/davidcolarusso/opt/anaconda3/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='../data/raw/www.courts.ca.gov/forms/e2917df3dd9a497daf3a29ca49782672.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


Downloading: https://www.courts.ca.gov/documents/wv800info.pdf


/Users/davidcolarusso/opt/anaconda3/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='../data/raw/www.courts.ca.gov/forms/bb9e239f96230c44aa199655f2e0988d.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


,id,jurisdiction,source,title,group,url,filename,downloaded
0,a895e7c54d5a41a458aef57e16da5f1d,CA,www.courts.ca.gov,How to Adopt a Child in California,ADOPT,https://www.courts.ca.gov/documents/adopt050in...,adopt050info.pdf,2021-11-17
1,3979f1c1c9f165ccac026b26cf20252c,CA,www.courts.ca.gov,Adoption Request,ADOPT,https://www.courts.ca.gov/documents/adopt200.pdf,adopt200.pdf,2021-11-17
2,9f441bfaca115775970cf03550e0c78e,CA,www.courts.ca.gov,Declaration Confirming Parentage in Stepparent...,ADOPT,https://www.courts.ca.gov/documents/adopt205.pdf,adopt205.pdf,2021-11-17
3,72e33ceb8675e0b13a39d488ffd2de48,CA,www.courts.ca.gov,Declaration Confirming Parentage in Stepparent...,ADOPT,https://www.courts.ca.gov/documents/adopt206.pdf,adopt206.pdf,2021-11-17
4,f020836e1802974aa0f6b2d6b8be5bb5,CA,www.courts.ca.gov,Adoption Agreement,ADOPT,https://www.courts.ca.gov/documents/adopt210.pdf,adopt210.pdf,2021-11-17
...,...,...,...,...,...,...,...,...
1473,6efc9088604befed5fb59fc42aeee0f5,CA,www.courts.ca.gov,Notice of Hearing to Renew Restraining Order,WV,https://www.courts.ca.gov/documents/wv710.pdf,wv710.pdf,2021-11-17
1474,740afc14f63fc6bf594ee344126ddb54,CA,www.courts.ca.gov,Response to Request to Renew Restraining Order,WV,https://www.courts.ca.gov/documents/wv720.pdf,wv720.pdf,2021-11-17
1475,c3642c835a4c4c0a167fdcc4758b68d9,CA,www.courts.ca.gov,Order Renewing Workplace Violence Restraining ...,WV,https://www.courts.ca.gov/documents/wv730.pdf,wv730.pdf,2021-11-17
1476,e2917df3dd9a497daf3a29ca49782672,CA,www.courts.ca.gov,"Proof of Firearms Turned In, Sold, or Stored",WV,https://www.courts.ca.gov/documents/wv800.pdf,wv800.pdf,2021-11-17


In [11]:
#download_pdf("https://www.courts.ca.gov/documents/adopt050info.pdf","../data/raw/www.courts.ca.gov/forms/","adopt050info.pdf")
#download_pdf("https://www.courts.ca.gov/documents/adopt205.pdf","../data/raw/www.courts.ca.gov/forms/","adopt205.pdf")

'error'

In [16]:
files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")

[back to top](#Jump-to-Jurisdiction)

# Download ME forms and meta data from

https://mjbportal.courts.maine.gov/CourtForms/FormsLists/Index

In [65]:
jur = "ME"
source = "mjbportal.courts.maine.gov"
form_path = "../data/raw/mjbportal.courts.maine.gov/forms/"
csv_path = "../data/raw/mjbportal.courts.maine.gov/"

today = date.today().strftime("%Y-%m-%d")

try:
    files_df = pd.read_csv(csv_path+"form_data.csv")
except:
    files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"])
    files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")
    
files_df[:1]

,id,jurisdiction,source,title,group,url,filename,downloaded


In [66]:
def ME_get(file):
    global files_df

    group = reCase(re.search(".*/([^/]*)\.html",file, re.IGNORECASE).group(1))

    with open(file) as f:
        contents = f.read()
    f.close()
    soup = BeautifulSoup(contents)
    body = soup.find('table',id="FormsListTbl")

    i = 0
    limit = 1000
    for a in body.find_all('a', href=True):
        
        url = re.search(".*/([^/]*)$",a['href'], re.IGNORECASE)
        filename = a.text.strip()+".pdf"
        fileurl = "https://mjbportal.courts.maine.gov"+url.group(0)

        title = re.search("%s\s+</a>\s+</td>\s+<td>([^<]*)"%(a.text.strip()),body.prettify(), re.IGNORECASE).groups()[0].strip()
        
        #print(fileurl)
        fileid = hashme(fileurl) #uuid.uuid4().hex

        #print("\t",title,group,filename,"\n\t",fileurl,"\n")

        status = ""
        if fileurl not in files_df["url"].values:
            print("Downloading: "+fileurl)
            if (i < limit):
                status = download_pdf(fileurl,form_path,fileid+".pdf")

            if (i < limit):
                files_df = files_df.append(pd.DataFrame([[fileid,jur,source,title,group,fileurl,filename,today]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"]), ignore_index=True,sort=False)
                i += 1
            else:
                print("skip")
            #time.sleep(1)

In [67]:
path = "../data/raw/mjbportal.courts.maine.gov/html/"
f = []
for (dirpath, dirnames, filenames) in walk(path):
    f.extend(filenames)
    break
f1 = [file for file in f]

i = 0
for file in f1:
    if re.search(".*\.html",file):# and i<1:
        print(i,path+file)
        ME_get(path+file)
        i+=1
        
files_df

0 ../data/raw/mjbportal.courts.maine.gov/html/screen02.html
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CV-061
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CV-066
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CV-067
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CV-072
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CV-092
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CV-100
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CV-145
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CV-172
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CV-173
Down

/Users/davidcolarusso/opt/anaconda3/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='../data/raw/mjbportal.courts.maine.gov/forms/8b554622b5701279167ed5cd2ee576a9.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=FM-006
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=FM-008


/Users/davidcolarusso/opt/anaconda3/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='../data/raw/mjbportal.courts.maine.gov/forms/e969f115f4dcadc4a3d39a0b40c04e4a.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=FM-020


...

/Users/davidcolarusso/opt/anaconda3/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='../data/raw/mjbportal.courts.maine.gov/forms/a56bf906d338198601a5ca9b327c8c35.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CR-248
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CR-249
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CR-250
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CR-251
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CR-CV-214
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CR-CV-FM-199
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CR-CV-FM-255
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CR-CV-FM-JV-021
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CR-CV-FM-JV-166
Downloading: https://mjbportal.

/Users/davidcolarusso/opt/anaconda3/lib/python3.8/site-packages/pdfminer/pdfpage.py:140: PDFTextExtractionNotAllowedWarning: The PDF <_io.BufferedReader name='../data/raw/mjbportal.courts.maine.gov/forms/46e518f044a923b8742a3cd655537a2d.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding.
  warnings.warn(warning_msg, PDFTextExtractionNotAllowedWarning)


Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CR-JV-253
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CV-001
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CV-007
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CV-037
Downloading: https://mjbportal.courts.maine.gov/CourtForms/FormsLists/DownloadForm?strFormNumber=CV-060


,id,jurisdiction,source,title,group,url,filename,downloaded
0,556887ab5f12a3dc9741e1cdf9210294,ME,mjbportal.courts.maine.gov,Affidavit &amp; Request for Default and Defaul...,screen 02,https://mjbportal.courts.maine.gov/CourtForms/...,CV-061.pdf,2021-11-16
1,a68e76839890044b2e119acb9ea9c416,ME,mjbportal.courts.maine.gov,Petition &amp; Order for Military Certification,screen 02,https://mjbportal.courts.maine.gov/CourtForms/...,CV-066.pdf,2021-11-16
2,93e5e2b67986ae54a44355978a3a4b8e,ME,mjbportal.courts.maine.gov,Application to Proceed Without Payment of Fees,screen 02,https://mjbportal.courts.maine.gov/CourtForms/...,CV-067.pdf,2021-11-16
3,7e427316c6a0616da294ca0664829911,ME,mjbportal.courts.maine.gov,Motion for Service by Alternate Means and Affi...,screen 02,https://mjbportal.courts.maine.gov/CourtForms/...,CV-072.pdf,2021-11-16
4,24d97bb89a3da0c9fc79ac110b151772,ME,mjbportal.courts.maine.gov,How to File a Civil Appeal,screen 02,https://mjbportal.courts.maine.gov/CourtForms/...,CV-092.pdf,2021-11-16
...,...,...,...,...,...,...,...,...
213,7db3ccf8291cc25aadee526d17862ac0,ME,mjbportal.courts.maine.gov,Notification That Observation is No Longer Nec...,screen 01,https://mjbportal.courts.maine.gov/CourtForms/...,CR-JV-253.pdf,2021-11-16
214,baec88eb3268a0754d3c1eb949acc1fe,ME,mjbportal.courts.maine.gov,Civil Summary Sheet,screen 01,https://mjbportal.courts.maine.gov/CourtForms/...,CV-001.pdf,2021-11-16
215,5ccfa502e8a951986add0248ea9a2bef,ME,mjbportal.courts.maine.gov,Complaint for Residential Forcible Entry and D...,screen 01,https://mjbportal.courts.maine.gov/CourtForms/...,CV-007.pdf,2021-11-16
216,aeb7825a71b2327c6b61676e83042fac,ME,mjbportal.courts.maine.gov,Subpoena on Motion for Contempt (*Self-represe...,screen 01,https://mjbportal.courts.maine.gov/CourtForms/...,CV-037.pdf,2021-11-16


In [68]:
files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")

[back to top](#Jump-to-Jurisdiction)

# Download UT forms and meta data from

Download forms and place them in the folder defined by `form_path` while creating a dataframe that you save to `csv_path`.


In [16]:
jur = "UT"
source = "www.utcourts.gov"
site = "https://www.utcourts.gov/forms/"
form_path = "../data/raw/www.utcourts.gov/forms/"
csv_path = "../data/raw/www.utcourts.gov/"

try:
    files_df = pd.read_csv(csv_path+"form_data.csv")
except:
    files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"])
    files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")   
    
files_df[:1]

,id,jurisdiction,source,title,group,url,filename,downloaded


In [17]:
with urllib.request.urlopen(urllib.request.Request(site)) as url:
    content = url.read().decode()
soup = BeautifulSoup(content)

In [18]:
today = date.today().strftime("%Y-%m-%d")
body = soup.find('div',id="content")
i = 0
limit = 5000
for section in re.split("<h3>",str(body), flags=re.IGNORECASE):
    group = re.match('^[^<]*',section)[0]
    
    #print(group)
    
    section_html = BeautifulSoup(section)
    for li in section_html.find_all('li'):
        title = re.sub('\(.*\s*.*','',li.text)
        
        #print("\t",i,title)
        
        for a in li.find_all('a', href=True):
            if (re.search("^http.*\.pdf",a['href'])):
                url = re.search(".*/([^/]*)\.pdf",a['href'], re.IGNORECASE)
                filename = url.group(1)+".pdf"
                fileurl = url.group(0)
                fileid = hashme(fileurl) #uuid.uuid4().hex
                
                #print("\t",filename,"\n\t",fileurl,"\n")
                
                status = ""
                if fileurl not in files_df["url"].values:
                    print("Downloading: "+fileurl)
                    if (i < limit):
                        status = download_pdf(fileurl,form_path,fileid+".pdf")

                    if status == "error":
                        print("#######################")
                        print("         Error!!!")
                        print("#######################")
                        #time.sleep(5)
                    else:
                        if (i < limit):
                            files_df = files_df.append(pd.DataFrame([[fileid,jur,source,title,group,fileurl,filename,today]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"]), ignore_index=True,sort=False)
                            i += 1
                        else:
                            print("skip")
                        #time.sleep(1)

Downloading: https://www.utcourts.gov/courts/juv/juvsites/3rd/forms/COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20BACK.pdf
Downloading: https://www.utcourts.gov/courts/juv/juvsites/3rd/forms/Work_Crew_Application-2007.pdf
Downloading: https://www.utcourts.gov/courts/juv/juvsites/3rd/common/Probation%20Order%20revised.050502.pdf
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/02_Consent_Adoptee.pdf
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/08_Agreement.pdf
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/10_Decree.pdf
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/03_1_Consent_Birth_Mother.pdf
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/07_Certificate_Readiness.pdf
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/docs/13_Motion_Alternative_Service.pdf
Downloading: https://www.utcourts.gov/howto/family/adoption/stepchild/doc

Downloading: https://www.utcourts.gov/howto/changing_lawyers/docs/1123GE_Notice_of_Personal_Appearance_or_Appointment_of_Counsel.pdf
Downloading: https://www.utcourts.gov/howto/changing_lawyers/docs/1121GE_Notice_of_Withdrawal_of_Counsel.pdf
Downloading: https://www.utcourts.gov/howto/changing_lawyers/docs/1122GE_Notice_to_Appear_Personally_or_to_Appoint_Counsel.pdf
Downloading: https://www.utcourts.gov/howto/changing_lawyers/docs/1124GE_Substitution_of_Counsel.pdf
Downloading: https://www.utcourts.gov/resources/forms/protectorder/docs/CPO-Child_Protective_Order.pdf
Downloading: https://www.utcourts.gov/resources/forms/protectorder/docs/CPO-Ex_Parte_Order.pdf
Downloading: https://www.utcourts.gov/howto/filing/motions/docs/1101JU_Juvenile_Court_Motion.pdf
Downloading: https://www.utcourts.gov/resources/forms/protectorder/docs/CPO-Notice_of_Denial.pdf
Downloading: https://www.utcourts.gov/resources/forms/protectorder/docs/transfer_child.pdf
Downloading: https://www.utcourts.gov/resources

Downloading: https://www.utcourts.gov/howto/divorce/docs/1250GE_Certification_of_Readiness_for_Trial.pdf
Downloading: https://www.utcourts.gov/howto/divorce/docs/1051FA_Declaration_of_Jurisdiction_and_Grounds_for_Divorce.pdf
Downloading: https://www.utcourts.gov/howto/divorce/docs/1052FA_Income_Verification_and_Compliance_with_Child_Support_Guidelines.pdf
Downloading: https://www.utcourts.gov/howto/divorce/docs/OCAP_Respondent's_Declaration_of_Military_Service.pdf
Downloading: https://www.utcourts.gov/howto/divorce/docs/Divorce_Certificate.pdf
Downloading: https://www.utcourts.gov/howto/divorce/docs/1261GE_Trial_Issues_Domestic_Cases.pdf
Downloading: https://www.utcourts.gov/mediation/divmed/forms/APPLICATION_TO_BE_EXCUSED.pdf
Downloading: https://www.utcourts.gov/mediation/divmed/forms/LOW_INCOME_SURVEY_WITH_DOB.pdf
Downloading: https://www.utcourts.gov/howto/family/domestic_relation_injunction/docs/1501FA_Domestic_Relations_Injunction.pdf
Downloading: https://www.utcourts.gov/resourc

Downloading: https://www.utcourts.gov/gramp/gsp/docs/Request%20to%20Appoint%20an%20Attorney%20to%20Represent%20the%20Respondent.pdf
Downloading: https://www.utcourts.gov/resources/forms/garnishment/docs/1503DC_Application_for_Writ_of_Garnishment.pdf
Downloading: https://www.utcourts.gov/resources/forms/garnishment/docs/1510DC_Motion_to_Enforce_Writ_of_Garnishment.pdf
Downloading: https://www.utcourts.gov/resources/forms/garnishment/docs/1510DC-C_Motion_to_Enforce_Writ_of_Garnishment-Commissioner.pdf
Downloading: https://www.utcourts.gov/resources/forms/garnishment/docs/1507DC_Garnishee_Answers_to_Interrogatories_for_Earnings.pdf
Downloading: https://www.utcourts.gov/resources/forms/garnishment/docs/1506DC_Garnishee_Answers_to_Interrogatories_for_Property_other_than_Earnings.pdf
Downloading: https://www.utcourts.gov/howto/judgment/docs/1004DC_Judgment_Information_Statement.pdf
Downloading: https://www.utcourts.gov/resources/forms/garnishment/docs/1508DC_Notice_of_Garnishment_and_Exempti

Downloading: https://www.utcourts.gov/howto/filing/motions/docs/1108GE_Affidavit_with_Exhibits.pdf
Downloading: https://www.utcourts.gov/howto/filing/motions/docs/1103GE_Counter_Motion.pdf
Downloading: https://www.utcourts.gov/howto/filing/motions/docs/1109GE_Exhibit_Summary.pdf
Downloading: https://www.utcourts.gov/howto/family/modification/child_support/docs/petition/1136FA_Findings_and_Conclusions_on_Petition_to_Modify_Child_Support.pdf
Downloading: https://www.utcourts.gov/howto/family/modification/child_support/docs/motion/1138FA_Motion_to_Adjust_Child_Support.pdf
Downloading: https://www.utcourts.gov/howto/family/modification/child_support/docs/motion/1138FA-C_Motion_to_Adjust_Child_Support-Commissioner.pdf.pdf
#######################
         Error!!!
#######################
Downloading: https://www.utcourts.gov/howto/family/modification/child_support/docs/motion/1138FA-C_Motion_to_Adjust_Child_Support-Commissioner.pdf
Downloading: https://www.utcourts.gov/howto/filing/non-publi

Downloading: https://www.utcourts.gov/howto/criminallaw/docs/1501CR_Motion_to_Remove_Link_to_Personal_Identifying_Information.pdf
Downloading: https://www.utcourts.gov/howto/criminallaw/docs/1502CR_Order_on_Motion_to_Remove_Link_to_Personal.pdf
Downloading: https://www.utcourts.gov/howto/judgment/renew/docs/1103DC_Findings_Conclusions_Order_Motion_to_Renew_Judgment.pdf
Downloading: https://www.utcourts.gov/howto/judgment/renew/docs/1102DC_Memorandum_Opposing_Motion_to_Renew_Judgment.pdf
Downloading: https://www.utcourts.gov/howto/judgment/renew/docs/1101DC_Motion_and_Affidavit_to_Renew_Judgment.pdf
Downloading: https://www.utcourts.gov/howto/judgment/set_aside/docs/1070GE_Motion_to_Set_Aside_Default_or_Judgment.pdf
Downloading: https://www.utcourts.gov/howto/judgment/set_aside/docs/1071GE_Order_on_Motion_to_Set_Aside_Default_or_Judgment.pdf
Downloading: https://www.utcourts.gov/howto/filing/motions/dl_suspension/docs/05_Abstract_of_Records.pdf
Downloading: https://www.utcourts.gov/howt

Downloading: https://www.utcourts.gov/howto/family/fact_of_birth/docs/01_Petition_for_Order_Establishing_Fact_of_Birth_Minor.pdf
Downloading: https://www.utcourts.gov/howto/petitiondcfs/docs/02_Order_on_Petition_for_Removal_from_DCFS_Custody.pdf
Downloading: https://www.utcourts.gov/howto/petitiondcfs/docs/04_Order_on_Petition_to_Re-Enter_DCFS_Custody.pdf
Downloading: https://www.utcourts.gov/howto/petitiondcfs/docs/01_Petition_for_Removal_from_DCFS_Custody.pdf
Downloading: https://www.utcourts.gov/howto/petitiondcfs/docs/03_Petition_to_Re-Enter%20DCFS_Custody.pdf
Downloading: https://www.utcourts.gov/media/docs/GRAMA_Request.pdf
#######################
         Error!!!
#######################
Downloading: https://www.utcourts.gov/howto/family/power_of_attorney_general/docs/02_Agent's_Certification.pdf
Downloading: https://www.utcourts.gov/howto/family/power_of_attorney_general/docs/01_Power_of_Attorney.pdf
Downloading: https://www.utcourts.gov/howto/family/power_of_attorney_general/d

Downloading: https://www.utcourts.gov/howto/family/enforcement/ors/docs/1261FA_Order_Confirming_Registration_of_ORS_Support_Order.pdf
Downloading: https://www.utcourts.gov/howto/family/enforcement/ors/docs/1260FA_Petition_to_Register_ORS_Support_Order.pdf
Downloading: https://www.utcourts.gov/howto/family/gc/docs/GC07_Financial_Accounting.pdf
Downloading: https://www.utcourts.gov/howto/family/gc/docs/GC09_Certificate_of_Mailing.pdf
Downloading: https://www.utcourts.gov/howto/family/gc/docs/GC06_Inventory.pdf
Downloading: https://www.utcourts.gov/howto/family/gc/docs/GC08_Notice_of_Right_to_Object.pdf
Downloading: https://www.utcourts.gov/howto/family/gc/docs/GC05_Report_on_Status_of_the_Ward.pdf
Downloading: https://www.utcourts.gov/resources/interp/request/docs/01_Request_Interpreter_Generic.pdf
Downloading: https://www.utcourts.gov/resources/interp/request/docs/01_Request_Interpreter_Spa.pdf
Downloading: https://www.utcourts.gov/resources/interp/request/docs/01_Request_Interpreter_Vi

Downloading: https://www.utcourts.gov/courts/dist/distsites/7th/docs/Family_Drug_Court_Motion.pdf
Downloading: https://www.utcourts.gov/courts/dist/distsites/7th/docs/Carbon%20County%20Juvenile%20Court%20Declaration%20of%20Financial%20Status.pdf
Downloading: https://www.utcourts.gov/courts/dist/distsites/7th/docs/Statement%20of%20Defendant%20Carbon%20County.pdf
Downloading: https://www.utcourts.gov/courts/dist/distsites/7th/docs/Emery%20County%20Adult%20Drug%20Court%20Motion.pdf
Downloading: https://www.utcourts.gov/courts/dist/distsites/7th/docs/Emery%20Affidavit%20of%20Indigency.pdf
Downloading: https://www.utcourts.gov/courts/dist/distsites/7th/docs/Emery%20County%20Criminal%20Motion.pdf
Downloading: https://www.utcourts.gov/courts/dist/distsites/7th/docs/Emery%20County%20Juvenile%20Court%20Declaration%20of%20Financial%20Status.pdf
Downloading: https://www.utcourts.gov/courts/dist/distsites/7th/docs/Statement%20of%20Defendant%20Emery%20County.pdf
Downloading: https://www.utcourts.go

In [19]:
display(files_df)

,id,jurisdiction,source,title,group,url,filename,downloaded
0,04b3a0734774c02edf8eb9056d23954aa38e96c77c3392...,UT,www.utcourts.gov,Community Service Worksheet Third District Juv...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,COMMUNITY%20SERVICE%20WORKSHEET-FRONT%20AND%20...,2021-11-11
1,6e420f1b3575cfd8ef94b71977da9e38252e3395a78439...,UT,www.utcourts.gov,Third District Juvenile Court Work Program Ref...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Work_Crew_Application-2007.pdf,2021-11-11
2,2532cd2b6d3aaff8c47726a0abd168fb4e5cdb4977c065...,UT,www.utcourts.gov,Utah State District Juvenile Court Probation O...,3rd District Juvenile Court: Forms and Pamphlets,https://www.utcourts.gov/courts/juv/juvsites/3...,Probation%20Order%20revised.050502.pdf,2021-11-11
3,f6a6814890f21c11524d5785d772272916ff95909b7dba...,UT,www.utcourts.gov,Adoptee's Consent to Adoption and Waiver of Ri...,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,02_Consent_Adoptee.pdf,2021-11-11
4,be656a9a361db7c3532026b0bae372e704beb160fce37e...,UT,www.utcourts.gov,Adoption Agreement,Adopting a Minor Stepchild,https://www.utcourts.gov/howto/family/adoption...,08_Agreement.pdf,2021-11-11
...,...,...,...,...,...,...,...,...
651,9e19a44392bdd3dd0c1a2ef88b84c75878819dda60f18c...,UT,www.utcourts.gov,Civil Wrongful Lien Injunction,Wrongful Liens,https://www.utcourts.gov/resources/forms/wrong...,05_Wrongful_Lien_Injunction.pdf,2021-11-11
652,74495f5ac80edf067e7bf609805e30ef1e67974fbf6d44...,UT,www.utcourts.gov,Ex Parte Civil Wrongful Lien Injunction,Wrongful Liens,https://www.utcourts.gov/resources/forms/wrong...,02_Ex_Parte_Wrongful_Lien_Injunction.pdf,2021-11-11
653,a8a88c0d521c7276703a8aa1e1edfea3a14c70fe1164f3...,UT,www.utcourts.gov,Order Revoking Ex Parte Civil Wrongful Lien In...,Wrongful Liens,https://www.utcourts.gov/resources/forms/wrong...,06_Order_Revoking_Ex_Parte_Injunction.pdf,2021-11-11
654,821243f1187eda4dde628ce1d50f08d7d87b59434deb18...,UT,www.utcourts.gov,Petition for Civil Wrongful Lien Injunction,Wrongful Liens,https://www.utcourts.gov/resources/forms/wrong...,01_Petition_for_Wrongful_Lien.pdf,2021-11-11


In [20]:
files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")

In [ ]:
# maybe delet old before copy
#os.system('cp ../data/raw/www.utcourts.gov/forms/* ../data/processed/www.utcourts.gov/forms/')

Now you have to go and auto-generate fields. 

[back to top](#Jump-to-Jurisdiction)

# Download MI forms and meta data from

Download forms and place them in the folder defined by `form_path` while creating a dataframe that you save to `csv_path`.


In [21]:
#from requests_html import HTMLSession
#from requests_html import AsyncHTMLSession
#import nest_asyncio
#import pyppdf.patch_pyppeteer

#asession = AsyncHTMLSession()
#r = await asession.get("https://www.courts.michigan.gov/SCAO-forms/appeals/")
#await r.html.arender(timeout=15, sleep=5)

In [22]:
jur = "MI"
source = "www.courts.michigan.gov"
form_path = "../data/raw/www.courts.michigan.gov/forms/"
csv_path = "../data/raw/www.courts.michigan.gov/"

today = date.today().strftime("%Y-%m-%d")

try:
    files_df = pd.read_csv(csv_path+"form_data.csv")
except:
    files_df = pd.DataFrame([],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"])
    files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")
    
files_df[:1]

,id,jurisdiction,source,title,group,url,filename,downloaded


The MI court site is all js rendered. So, I downloaded the rendered pages into `../data/raw/www.courts.michigan.gov/html/` and will deal with them there. I used `copy(document.body.innerHTML);` in console to grab the rendered page.

In [23]:
def MI_get(file):
    global files_df

    group = reCase(re.search(".*/([^/]*)\.html",file, re.IGNORECASE).group(1))

    with open(file) as f:
        contents = f.read()
    f.close()
    soup = BeautifulSoup(contents)
    body = soup.find('div',id="main-content")

    i = 0
    limit = 1000
    for a in body.find_all('a', href=True):
        if (re.search(".*\.pdf",a['href'])):
            title = a.text.strip()
            url = re.search(".*/([^/]*)\.pdf",a['href'], re.IGNORECASE)
            filename = url.group(1)+".pdf"
            fileurl = "https://www.courts.michigan.gov"+url.group(0)
            fileid = hashme(fileurl) #uuid.uuid4().hex

            #print("\t",title,group,filename,"\n\t",fileurl,"\n")

            status = ""
            if fileurl not in files_df["url"].values:
                print("Downloading: "+fileurl)
                if (i < limit):
                    status = download_pdf(fileurl,form_path,fileid+".pdf")

                if status == "error":
                    print("#######################")
                    print("         Error!!!")
                    print("#######################")
                    time.sleep(5)
                else:
                    if (i < limit):
                        files_df = files_df.append(pd.DataFrame([[fileid,jur,source,title,group,fileurl,filename,today]],columns=["id","jurisdiction","source","title","group","url","filename","downloaded"]), ignore_index=True,sort=False)
                        i += 1
                    else:
                        print("skip")
                    #time.sleep(1)

In [24]:
path = "../data/raw/www.courts.michigan.gov/html/"
f = []
for (dirpath, dirnames, filenames) in walk(path):
    f.extend(filenames)
    break
f1 = [file for file in f]

for file in f1:
    if re.search(".*\.html",file):
        #print(path+file)
        MI_get(path+file)

Downloading: https://www.courts.michigan.gov/4a205c/siteassets/forms/scao-approved/dc111cnoinstructions.pdf
Downloading: https://www.courts.michigan.gov/4a7a08/siteassets/forms/scao-approved/dc111c.pdf
Downloading: https://www.courts.michigan.gov/4a2228/siteassets/forms/scao-approved/dc111bnoinstructions.pdf
Downloading: https://www.courts.michigan.gov/4a74bc/siteassets/forms/scao-approved/dc111b.pdf
Downloading: https://www.courts.michigan.gov/4a74ca/siteassets/forms/scao-approved/dc111a.pdf
Downloading: https://www.courts.michigan.gov/4a2499/siteassets/forms/scao-approved/dc111anoinstructions.pdf
Downloading: https://www.courts.michigan.gov/4a2499/siteassets/forms/scao-approved/dc111dnoinstructions.pdf
Downloading: https://www.courts.michigan.gov/4a79f3/siteassets/forms/scao-approved/dc111d.pdf
Downloading: https://www.courts.michigan.gov/4a20f0/siteassets/forms/scao-approved/dc107.pdf
Downloading: https://www.courts.michigan.gov/4a20fd/siteassets/forms/scao-approved/dc103.pdf
Downlo

Downloading: https://www.courts.michigan.gov/4a2200/siteassets/forms/scao-approved/pca306.pdf
Downloading: https://www.courts.michigan.gov/4a2127/siteassets/forms/scao-approved/pca305a.pdf
Downloading: https://www.courts.michigan.gov/4a2067/siteassets/forms/scao-approved/pca305.pdf
Downloading: https://www.courts.michigan.gov/4a21be/siteassets/forms/scao-approved/pca305i.pdf
Downloading: https://www.courts.michigan.gov/49a276/siteassets/forms/scao-approved/pca353.pdf
Downloading: https://www.courts.michigan.gov/4a2375/siteassets/forms/scao-approved/pca331.pdf
Downloading: https://www.courts.michigan.gov/49a21d/siteassets/forms/scao-approved/pca340.pdf
Downloading: https://www.courts.michigan.gov/4a2424/siteassets/forms/scao-approved/pca332a.pdf
Downloading: https://www.courts.michigan.gov/4ac533/siteassets/forms/scao-approved/pca329.pdf
Downloading: https://www.courts.michigan.gov/4a2140/siteassets/forms/scao-approved/pca330.pdf
Downloading: https://www.courts.michigan.gov/4aa295/sitea

Downloading: https://www.courts.michigan.gov/4a7b95/siteassets/forms/scao-approved/mc15a.pdf
Downloading: https://www.courts.michigan.gov/4a7bae/siteassets/forms/scao-approved/mc52.pdf
Downloading: https://www.courts.michigan.gov/4a808d/siteassets/forms/scao-approved/mc13.pdf
Downloading: https://www.courts.michigan.gov/4a7aed/siteassets/forms/scao-approved/mc12.pdf
Downloading: https://www.courts.michigan.gov/4a3c82/siteassets/forms/scao-approved/mc440.pdf
Downloading: https://www.courts.michigan.gov/4a27d3/siteassets/forms/scao-approved/mc62.pdf
Downloading: https://www.courts.michigan.gov/4a7dab/siteassets/forms/scao-approved/mc03.pdf
Downloading: https://www.courts.michigan.gov/4a7de0/siteassets/forms/scao-approved/mc315.pdf
Downloading: https://www.courts.michigan.gov/4a27ee/siteassets/forms/scao-approved/mc17.pdf
Downloading: https://www.courts.michigan.gov/4a2807/siteassets/forms/scao-approved/mc01a.pdf
Downloading: https://www.courts.michigan.gov/4a27a5/siteassets/forms/scao-ap

Downloading: https://www.courts.michigan.gov/4a7dd3/siteassets/forms/scao-approved/pc573.pdf
Downloading: https://www.courts.michigan.gov/4a8010/siteassets/forms/scao-approved/pc587.pdf
Downloading: https://www.courts.michigan.gov/4a7bb9/siteassets/forms/scao-approved/pc600.pdf
Downloading: https://www.courts.michigan.gov/4a7e8d/siteassets/forms/scao-approved/pc580.pdf
Downloading: https://www.courts.michigan.gov/4a7cce/siteassets/forms/scao-approved/pc619.pdf
Downloading: https://www.courts.michigan.gov/4a7b27/siteassets/forms/scao-approved/pc562.pdf
Downloading: https://www.courts.michigan.gov/4a2ad6/siteassets/forms/scao-approved/pc689.pdf
Downloading: https://www.courts.michigan.gov/4a7dfd/siteassets/forms/scao-approved/pc575.pdf
Downloading: https://www.courts.michigan.gov/4a8039/siteassets/forms/scao-approved/pc589.pdf
Downloading: https://www.courts.michigan.gov/4a8011/siteassets/forms/scao-approved/pc557.pdf
Downloading: https://www.courts.michigan.gov/4a7e4a/siteassets/forms/s

Downloading: https://www.courts.michigan.gov/4a8156/siteassets/forms/scao-approved/cc375.pdf
Downloading: https://www.courts.michigan.gov/4a7f2a/siteassets/forms/scao-approved/cc395.pdf
Downloading: https://www.courts.michigan.gov/4a8163/siteassets/forms/scao-approved/cc377m.pdf
Downloading: https://www.courts.michigan.gov/4a819b/siteassets/forms/scao-approved/cc395m.pdf
Downloading: https://www.courts.michigan.gov/4a816a/siteassets/forms/scao-approved/cc375m.pdf
Downloading: https://www.courts.michigan.gov/4a7f1e/siteassets/forms/scao-approved/cc386.pdf
Downloading: https://www.courts.michigan.gov/4a7f71/siteassets/forms/scao-approved/mc319.pdf
Downloading: https://www.courts.michigan.gov/4a2f51/siteassets/forms/scao-approved/pc51b.pdf
Downloading: https://www.courts.michigan.gov/4aa6ee/siteassets/forms/scao-approved/pc52.pdf
Downloading: https://www.courts.michigan.gov/4a7f82/siteassets/forms/scao-approved/pc51.pdf
Downloading: https://www.courts.michigan.gov/4a721c/siteassets/forms/

Downloading: https://www.courts.michigan.gov/4aa5f5/siteassets/forms/scao-approved/jc95.pdf
Downloading: https://www.courts.michigan.gov/4a2bb5/siteassets/forms/scao-approved/jc104.pdf
Downloading: https://www.courts.michigan.gov/4aa5f8/siteassets/forms/scao-approved/jc100.pdf
Downloading: https://www.courts.michigan.gov/4aa5f4/siteassets/forms/scao-approved/jc97.pdf
Downloading: https://www.courts.michigan.gov/4a2bb8/siteassets/forms/scao-approved/jc99o.pdf
Downloading: https://www.courts.michigan.gov/4a7ea7/siteassets/forms/scao-approved/jc98o.pdf
Downloading: https://www.courts.michigan.gov/4aa5e1/siteassets/forms/scao-approved/jc101.pdf
Downloading: https://www.courts.michigan.gov/4a2be0/siteassets/forms/scao-approved/jc99m.pdf
Downloading: https://www.courts.michigan.gov/4a7d9a/siteassets/forms/scao-approved/jc98m.pdf
Downloading: https://www.courts.michigan.gov/4a7d86/siteassets/forms/scao-approved/jc96.pdf
Downloading: https://www.courts.michigan.gov/4a7d5b/siteassets/forms/scao

Downloading: https://www.courts.michigan.gov/4a7ad1/siteassets/forms/scao-approved/jc78.pdf
Downloading: https://www.courts.michigan.gov/4a7fb1/siteassets/forms/scao-approved/jc77.pdf
Downloading: https://www.courts.michigan.gov/4a8036/siteassets/forms/scao-approved/jc06.pdf
Downloading: https://www.courts.michigan.gov/4a8218/siteassets/forms/scao-approved/jc23.pdf
Downloading: https://www.courts.michigan.gov/4a7e47/siteassets/forms/scao-approved/foc16.pdf
Downloading: https://www.courts.michigan.gov/4a7702/siteassets/forms/scao-approved/foc101.pdf
Downloading: https://www.courts.michigan.gov/4a7c32/siteassets/forms/scao-approved/foc63.pdf
Downloading: https://www.courts.michigan.gov/4a7282/siteassets/forms/scao-approved/foc125.pdf
Downloading: https://www.courts.michigan.gov/4a7e56/siteassets/forms/scao-approved/foc14.pdf
Downloading: https://www.courts.michigan.gov/4a81b1/siteassets/forms/scao-approved/foc4.pdf
Downloading: https://www.courts.michigan.gov/4a7297/siteassets/forms/scao

Downloading: https://www.courts.michigan.gov/4a6343/siteassets/forms/scao-approved/cc99e.pdf
Downloading: https://www.courts.michigan.gov/4a7c61/siteassets/forms/scao-approved/foc67.pdf
Downloading: https://www.courts.michigan.gov/4a7d11/siteassets/forms/scao-approved/foc82.pdf
Downloading: https://www.courts.michigan.gov/4a7a0a/siteassets/forms/scao-approved/foc110.pdf
Downloading: https://www.courts.michigan.gov/4a6399/siteassets/forms/scao-approved/cc389.pdf
Downloading: https://www.courts.michigan.gov/4a63a2/siteassets/forms/scao-approved/cc436.pdf
Downloading: https://www.courts.michigan.gov/4a7b7c/siteassets/forms/scao-approved/foc44.pdf
Downloading: https://www.courts.michigan.gov/4a7d29/siteassets/forms/scao-approved/foc86.pdf
Downloading: https://www.courts.michigan.gov/4a63e0/siteassets/forms/scao-approved/cc88.pdf
Downloading: https://www.courts.michigan.gov/4a7c2c/siteassets/forms/scao-approved/foc64.pdf
Downloading: https://www.courts.michigan.gov/4a7d19/siteassets/forms/s

Downloading: https://www.courts.michigan.gov/4a7e53/siteassets/forms/scao-approved/mc443.pdf
Downloading: https://www.courts.michigan.gov/4a7e46/siteassets/forms/scao-approved/mc399.pdf
Downloading: https://www.courts.michigan.gov/4aa817/siteassets/forms/scao-approved/dc255.pdf
Downloading: https://www.courts.michigan.gov/4a2a47/siteassets/forms/scao-approved/mc310.pdf
Downloading: https://www.courts.michigan.gov/4a2a75/siteassets/forms/scao-approved/mc283.pdf
Downloading: https://www.courts.michigan.gov/4aa6fa/siteassets/forms/scao-approved/mc394a.pdf
Downloading: https://www.courts.michigan.gov/4a2e9e/siteassets/forms/scao-approved/mc228b.pdf
Downloading: https://www.courts.michigan.gov/4a2e9f/siteassets/forms/scao-approved/mc228.pdf
Downloading: https://www.courts.michigan.gov/4a2e8c/siteassets/forms/scao-approved/mc228a.pdf
Downloading: https://www.courts.michigan.gov/4aa6e4/siteassets/forms/scao-approved/mc399a.pdf
Downloading: https://www.courts.michigan.gov/4a2a59/siteassets/for

Downloading: https://www.courts.michigan.gov/4a7d23/siteassets/forms/scao-approved/pcm208a.pdf
Downloading: https://www.courts.michigan.gov/4a7d30/siteassets/forms/scao-approved/pc627.pdf
Downloading: https://www.courts.michigan.gov/4a7e53/siteassets/forms/scao-approved/pc634.pdf
Downloading: https://www.courts.michigan.gov/4a7ea3/siteassets/forms/scao-approved/pc654.pdf
Downloading: https://www.courts.michigan.gov/4a7c73/siteassets/forms/scao-approved/pc686.pdf
Downloading: https://www.courts.michigan.gov/4a7f0a/siteassets/forms/scao-approved/pc633.pdf
Downloading: https://www.courts.michigan.gov/4a7c1a/siteassets/forms/scao-approved/pc662.pdf
Downloading: https://www.courts.michigan.gov/4a7b03/siteassets/forms/scao-approved/pc652.pdf
Downloading: https://www.courts.michigan.gov/4a7c33/siteassets/forms/scao-approved/pc670.pdf
Downloading: https://www.courts.michigan.gov/4a7e64/siteassets/forms/scao-approved/pc672.pdf
Downloading: https://www.courts.michigan.gov/4a7e35/siteassets/forms

In [25]:
display(files_df)

,id,jurisdiction,source,title,group,url,filename,downloaded
0,e12318ce9037e92ee8e48963f550e1a3229a17a2437b9e...,MI,www.courts.michigan.gov,Answer to Complaint to Recover Possession of P...,LTLC forms,https://www.courts.michigan.gov/4a205c/siteass...,dc111cnoinstructions.pdf,2021-11-11
1,eb4e35c3b9f355d63ad14ae71866d44b14e61196763553...,MI,www.courts.michigan.gov,Answer to Complaint to Recover Possession of P...,LTLC forms,https://www.courts.michigan.gov/4a7a08/siteass...,dc111c.pdf,2021-11-11
2,74723a905136a3e30a9eec9aa9191f5d3e4b800ae319df...,MI,www.courts.michigan.gov,"Answer, Damage/Health Hazard to Property, Land...",LTLC forms,https://www.courts.michigan.gov/4a2228/siteass...,dc111bnoinstructions.pdf,2021-11-11
3,ee10d4fd30c1faf309bd7b1a91da0794ec8862ee91c355...,MI,www.courts.michigan.gov,"Answer, Damage/Health Hazard to Property, Land...",LTLC forms,https://www.courts.michigan.gov/4a74bc/siteass...,dc111b.pdf,2021-11-11
4,5345e31f71a6ac7dc730e96039d35c9fc5d2cda658332e...,MI,www.courts.michigan.gov,"Answer, Nonpayment of Rent, Landlord-Tenant",LTLC forms,https://www.courts.michigan.gov/4a74ca/siteass...,dc111a.pdf,2021-11-11
...,...,...,...,...,...,...,...,...
867,31f0385280ba8218dbace7693ea04977c1cfa3278ec4f8...,MI,www.courts.michigan.gov,Claim of Appeal of Right/Request to Withdraw A...,civil infraction,https://www.courts.michigan.gov/4a2ca8/siteass...,cia05.pdf,2021-11-11
868,2a0847b00aa3dcf37dff8c1b2c3d4dcb3247445678cda1...,MI,www.courts.michigan.gov,"Default Judgment, Civil Infraction",civil infraction,https://www.courts.michigan.gov/4a7328/siteass...,cia07.pdf,2021-11-11
869,07e5f95f3f4042ecd5b58fcf8a141e8931ab84124a6c19...,MI,www.courts.michigan.gov,"Judgment, Civil Infraction",civil infraction,https://www.courts.michigan.gov/4a7336/siteass...,cia02.pdf,2021-11-11
870,15f70625faa5b1a09b1839d4003d806921983fea9a8b65...,MI,www.courts.michigan.gov,Motion to Set Aside Default Judgment and Order...,civil infraction,https://www.courts.michigan.gov/4a2cb4/siteass...,cia04.pdf,2021-11-11


In [26]:
files_df.to_csv(csv_path+"form_data.csv", index=False, encoding="utf-8")

In [ ]:
# maybe delet old before copy
#os.system('cp ../data/raw/www.courts.michigan.gov/forms/* ../data/processed/www.courts.michigan.gov/forms/')